In [1]:
import torch
import torch.nn.functional as F
import sys
sys.path.append("../models/")
from CustomCNNVessel import CustomResNet

def get_all_gradients(model, image, sampling_rate=1, device="cuda", vectorize=False):

    model.to(device)
    model = model.eval()
    model_wrapped = wrapper(model)
    sampled_image = image[:,::sampling_rate,::sampling_rate]
    sampled_image = sampled_image.to(device).unsqueeze(0)
    sampled_image.requires_grad = True

    jacobian = torch.autograd.functional.jacobian(model_wrapped, 
                                                            sampled_image,
                                                            vectorize = vectorize)
    jacobian = jacobian.squeeze().to('cpu')
    return jacobian

def wrapper(model):
    '''Wrap model to return probabilities and only the vessel channel.'''
    def new_model(img):
        out = model(img)
        probs = F.softmax(out, dim=1)
        return probs[:,1]
    return new_model

In [2]:
""" #Normal Vessels 
import os
from PIL import Image
import numpy as np

# Carregando as imagens
def load_images_from_directory(directory_name):
    image_files = sorted(os.listdir(directory_name))

    images = []
    for file_name in image_files:
        if file_name.endswith('.png'):
            img_path = os.path.join(directory_name, file_name)
            img = Image.open(img_path)
            img_array = np.array(img) / 255.0
            images.append(img_array)

    return torch.tensor(np.array(images), dtype=torch.float).unsqueeze(1).to('cpu') # Salvando as imagens na RAM porque a VRAM da gpu é liberada a cada cálculo
  
original_images = load_images_from_directory('./cropped_images')
original_images.shape """

" #Normal Vessels \nimport os\nfrom PIL import Image\nimport numpy as np\n\n# Carregando as imagens\ndef load_images_from_directory(directory_name):\n    image_files = sorted(os.listdir(directory_name))\n\n    images = []\n    for file_name in image_files:\n        if file_name.endswith('.png'):\n            img_path = os.path.join(directory_name, file_name)\n            img = Image.open(img_path)\n            img_array = np.array(img) / 255.0\n            images.append(img_array)\n\n    return torch.tensor(np.array(images), dtype=torch.float).unsqueeze(1).to('cpu') # Salvando as imagens na RAM porque a VRAM da gpu é liberada a cada cálculo\n  \noriginal_images = load_images_from_directory('./cropped_images')\noriginal_images.shape "

In [3]:
# Augmented Vessels

import os
from PIL import Image
import numpy as np

# Carregando as imagens
def load_images_from_directory(directory_name, image_list = []):
    image_files = sorted(os.listdir(directory_name))

    images = []
    for file_name in image_files:
        if file_name.endswith('.tiff'):
            img_path = os.path.join(directory_name, file_name)
            image_list.append(img_path)
            img = Image.open(img_path)
            img_array = np.array(img) / 255.0
            images.append(img_array)

    return torch.tensor(np.array(images), dtype=torch.float).unsqueeze(1).to('cpu'), image_list # Salvando as imagens na RAM porque a VRAM da gpu é liberada a cada cálculo
  
original_images,  image_list = load_images_from_directory("/home/fonta42/Desktop/interpretacao-redes-neurais/data/augmented_vessels")
original_images.shape

torch.Size([11, 1, 128, 128])

In [4]:
""" # instanciando o modelo
model_layers = (1, 1, 1)
model_channels = (64, 64, 64)
model = torchtrainer.models.resunet.ResUNetV2((8,), (1,), (64,))
# carregando o state_dict
baseline_state_dict = torch.load('./unetv2_baseline/checkpoint_best.pth')['model']
model.load_state_dict(baseline_state_dict) """

" # instanciando o modelo\nmodel_layers = (1, 1, 1)\nmodel_channels = (64, 64, 64)\nmodel = torchtrainer.models.resunet.ResUNetV2((8,), (1,), (64,))\n# carregando o state_dict\nbaseline_state_dict = torch.load('./unetv2_baseline/checkpoint_best.pth')['model']\nmodel.load_state_dict(baseline_state_dict) "

In [5]:
import torchseg

unet_restnet_model = torchseg.Unet(
    encoder_name="resnet18",
    encoder_weights=True,
    in_channels=1,
    classes=2,
).to('cuda')

unet_restnet_model.load_state_dict(torch.load("/home/fonta42/Desktop/interpretacao-redes-neurais/models/torchseg/pretrained_unet_resnet.pt"))

<All keys matched successfully>

In [7]:
import time

for idx in range(11):
    start_time = time.time()  # Record the start time before extracting the gradient
    
    gradient = get_all_gradients(unet_restnet_model, 
                                 original_images[idx], 
                                 sampling_rate=1)
    print(gradient.shape)
    print(torch.max(gradient))
    break
    # Save the gradient
    torch.save(gradient, f'./torchseg_gradients/jacobian_gradient_{idx}.pt')
    
    end_time = time.time()  # Record the end time after saving the gradient
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    print(f"Gradient {idx} extraction and saving took {elapsed_time:.2f} seconds.")
    
    break

torch.Size([1, 1, 128, 128])


KeyboardInterrupt: 

In [ ]:
import torchseg

unet_restnet_model = torchseg.Unet(
    encoder_name="resnet18",
    encoder_weights=False,
    in_channels=1,
    classes=2,
).to('cuda')

unet_restnet_model.load_state_dict(torch.load("/home/fonta42/Desktop/interpretacao-redes-neurais/models/torchseg/unet_resnet.pt"))

import time

for idx in range(11):
    start_time = time.time()  # Record the start time before extracting the gradient
    
    gradient = get_all_gradients(unet_restnet_model, 
                                 original_images[idx], 
                                 sampling_rate=1)
    
    # Save the gradient
    torch.save(gradient, f'./torchseg_gradients/jacobian_gradient_{idx}.pt')
    
    end_time = time.time()  # Record the end time after saving the gradient
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    
    print(f"Gradient {idx} extraction and saving took {elapsed_time:.2f} seconds.")

Gradient 0 extraction and saving took 40.11 seconds.
Gradient 1 extraction and saving took 38.89 seconds.
Gradient 2 extraction and saving took 39.26 seconds.
Gradient 3 extraction and saving took 39.24 seconds.
Gradient 4 extraction and saving took 38.43 seconds.
Gradient 5 extraction and saving took 38.53 seconds.
Gradient 6 extraction and saving took 38.77 seconds.
Gradient 7 extraction and saving took 38.65 seconds.
Gradient 8 extraction and saving took 38.97 seconds.
Gradient 9 extraction and saving took 41.89 seconds.
Gradient 10 extraction and saving took 40.91 seconds.


In [ ]:
model_weighted = CustomResNet(num_classes=2)
model_weighted.load_state_dict(torch.load(f"../models/vess_map_regularized_none_200.pth"))
for idx in range(4,11):
  gradient = get_all_gradients(model_weighted, 
                               original_images[idx], 
                               sampling_rate = 1)
  
  # Salvando o gradiente
  torch.save(gradient, f'./gradients_augmented_vessels/jacobian_gradient_{idx}.pt')

KeyboardInterrupt: 

In [ ]:
loaded_gradient = torch.load(f'./gradients/jacobian_gradient_{1}.pt')
loaded_gradient.shape

torch.Size([64, 64, 64, 64])

In [ ]:
rel_diff = ((loaded_gradient
 - jacobian)/jacobian).abs()
max_diff = rel_diff[jacobian.abs()>1e-2].max()
print(max_diff)

tensor(6746.3711)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
class ImageUF:
    '''Represents a float tensor as uint8, storing the minimum and
    maximum values.'''

    def __init__(self, image=None):

        if image is None:
            self.min = None
            self.max = None
            self.image_uint8 = None
        else:
            min = image.min()
            max = image.max()

            image_norm = 255.*(image - min)/(max - min)
            image_uint8 = image_norm.round().to(torch.uint8)
            
            self.min = min
            self.max = max
            self.image_uint8 = image_uint8

    def state_dict(self):

        state = {
            'min': self.min,
            'max': self.max,
            'image_uint8': self.image_uint8
        }

        return state
    
    def load_state_dict(self, state):

        self.min = state['min']
        self.max = state['max']
        self.image_uint8 = state['image_uint8']

    def to_float(self):

        min, max = self.min, self.max
        image_float32_norm = self.image_uint8.to(torch.float32)
        image_float32 = image_float32_norm*(max-min)/255. + min

        return image_float32

jacobian_uf = ImageUF(jacobian)
# Save data
torch.save(jacobian_uf.state_dict(), 'jacobian.pt')

In [ ]:
# Load data form disk
jacobian_uf2 = ImageUF()
jacobian_uf.load_state_dict(torch.load('jacobian.pt'))
# Convert to float again
jacobian_rec = jacobian_uf.to_float()

# Check maximum relative difference between values larger than 0.01
rel_diff = ((jacobian_rec - jacobian)/jacobian).abs()
max_diff = rel_diff[jacobian.abs()>1e-2].max()
print(max_diff)

tensor(0.0279)
